In [1]:
import json
import urllib.request, json 
import pandas as pd
import os
import functools
import time
import sys
from env import *

The following function is taken from https://realpython.com/primer-on-python-decorators/ and is used to measure the runtime of functions.

In [2]:
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

Helper function for flatting multidimensional list

In [3]:
def flatten(test_list):
    if isinstance(test_list, list):
        if len(test_list) == 0:
            return []
        first, rest = test_list[0], test_list[1:]
        return flatten(first) + flatten(rest)
    else:
        return [test_list]

Loading first 30 chars of every question, for identification of pageview

In [4]:
df = pd.read_excel(LIST_OF_QUESTIONS, sheet_name="Questions")
questions = df['First 30 chars']

Function for pairing pageviews to Limesurvey questions. Function loads baked session data and then for every pageview of this loaded session is loading raw pageview data. From raw pageview data are then extracted elements containing text of question displayed on pageview. This text is then compared to first 30 chars of every question and if pageview contains it, pageview is marked with number of page in Limesurvey and this pair is stored in dictionary. Function after checking all pageviews of session then returns pairing dictionary.

In [5]:
@timer
def pairing(path_to_files, session, questions):
    with open(path_to_files + session + ".json") as loadfile:
        data = json.load(loadfile)
    
    dict_pageviews = {}
    
    for pageview_count, pageview in enumerate(data['pageviews']):
        # print(pageview_count, pageview['id'])
        
        dict_pageviews[pageview['id']] = "unknown"
        
        path_to_specimen_pageview = path_to_files + session + "/" + pageview['id'] + ".json"

        with open(path_to_specimen_pageview) as loadfile:
            g_pageview = json.load(loadfile)
    
        for key in g_pageview:
            if key['type'] == 6:
                flat = flatten(key['args'])
                for count, f in enumerate(flat):
                    if isinstance(f, str):
                        for number, question in enumerate(questions):
                            if question in f:
                                # print("pageview", number)
                                dict_pageviews[pageview['id']] = number
                                break
    return dict_pageviews
        

Loading table of sessions and information about them.

In [6]:
df = pd.read_excel(PAIRED_SESSIONS, sheet_name="Sessions")
df = df.dropna(axis = 0, subset="Session")
df[:5]

,order,status,started at,corrected start at,device,os,browser,location,time taken,Session,Task,Limesurvey_Seed
0,1,completed,2022-03-27 19:58:19,2022-03-27 21:58:19,desktop,Windows,Chrome,SK,00:02:13,sMjVKK5bH9bGGEeDF4TMra2xb,2.0,-
1,3,completed,2022-03-27 20:00:45,2022-03-27 22:00:45,desktop,Windows,Chrome,SK,00:01:36,S5WLXErzUWNQqkDKnRC1j53PD,1.0,-
5,8,completed,2022-03-29 08:50:10,2022-03-29 10:50:10,desktop,Windows,Chrome,SK,00:13:58,gsd4FIhsnwaOIMOtoSqX4geVy,1.0,1445944018
6,9,abandoned,2022-03-29 08:55:04,2022-03-29 10:55:04,desktop,Windows,Chrome,SK,00:00:21,q7Cvsx4iO2KeFdZj7kRY2Izr5,1.0,NaN
8,11,completed,2022-03-29 09:14:01,2022-03-29 11:14:01,desktop,Windows,Chrome,SK,00:07:08,fjAPfBwmai5If7wWXOgoMldv5,2.0,1262028965


Dividing sessions to FG and H.

In [7]:
# Task 1 - FG
# Task 2 - H

fg_from_excel = df.loc[df['Task'] == 1]["Session"]
h_from_excel = df.loc[df['Task'] == 2]["Session"]

In [8]:
len(fg_from_excel) + len(h_from_excel) # should be 165

165

Function for marking sessions and saving results into json. Function is calling pairing function for every session from provided list of sessions. Results of pairing are then stored in dictionary. When all pageview of all sessions are paired and dictionary is complete, function saves dictionary into JSON.

In [9]:
def mark_sessions(sessions, path, questions):
    dict_sessions = {}
    for number, session in enumerate(sessions):
        print("Session", number + 1, "of", len(sessions))
        dict_sessions[session] = pairing(path, session, questions)
    with open(path + 'Sessions pairing.json', 'w') as fp:
        json.dump(dict_sessions, fp)
    return dict_sessions

Calling marking function for FG.

In [10]:
mark_sessions(fg_from_excel, FG_FILES, questions)

Session 1 of 91
Finished 'pairing' in 0.5834 secs
Session 2 of 91
Finished 'pairing' in 0.5490 secs
Session 3 of 91
Finished 'pairing' in 0.0066 secs
Session 4 of 91
Finished 'pairing' in 0.5397 secs
Session 5 of 91
Finished 'pairing' in 0.0160 secs
Session 6 of 91
Finished 'pairing' in 0.5041 secs
Session 7 of 91
Finished 'pairing' in 0.0819 secs
Session 8 of 91
Finished 'pairing' in 0.0210 secs
Session 9 of 91
Finished 'pairing' in 0.8429 secs
Session 10 of 91
Finished 'pairing' in 1.1143 secs
Session 11 of 91
Finished 'pairing' in 0.1221 secs
Session 12 of 91
Finished 'pairing' in 0.1188 secs
Session 13 of 91
Finished 'pairing' in 1.1089 secs
Session 14 of 91
Finished 'pairing' in 0.8313 secs
Session 15 of 91
Finished 'pairing' in 0.8197 secs
Session 16 of 91
Finished 'pairing' in 0.0175 secs
Session 17 of 91
Finished 'pairing' in 1.4524 secs
Session 18 of 91
Finished 'pairing' in 1.0184 secs
Session 19 of 91
Finished 'pairing' in 1.1086 secs
Session 20 of 91
Finished 'pairing' in 1

{'S5WLXErzUWNQqkDKnRC1j53PD': {'99aac600-ae08-11ec-8a65-4423fac019ed': 0,
  '9cafa190-ae08-11ec-9c60-98c476082d49': 1,
  '9d926070-ae08-11ec-adcc-cd540e47f2f1': 2,
  '9ef23e90-ae08-11ec-97ce-3544e0487fa7': 3,
  '9fa53ae0-ae08-11ec-88f7-b5f4787baadc': 4,
  'a0426540-ae08-11ec-a913-115307bf19f8': 5,
  'a0d4e140-ae08-11ec-978d-2c820dd76249': 6,
  'a16eb040-ae08-11ec-845c-79f93615f1e2': 7,
  'a2213760-ae08-11ec-8681-1f69d7436328': 8,
  'a2cae4e0-ae08-11ec-ac31-78f961c6a2e6': 9,
  'a3942850-ae08-11ec-be93-ea894e1a0ebf': 10,
  'a412ce30-ae08-11ec-82fe-bc3068a71df6': 10,
  'a5ade090-ae08-11ec-860b-b7336ef6f591': 11,
  'a65f0820-ae08-11ec-b8c1-d2f07f2e9afc': 12,
  'a739fed0-ae08-11ec-a571-9703760e7cbb': 13,
  'a7fbc830-ae08-11ec-baf3-4b73b9888899': 14,
  'a8982f40-ae08-11ec-8aa5-9b69cb1a40ad': 15,
  'a92ec9f0-ae08-11ec-b229-848e339127dd': 16,
  'a9d3e390-ae08-11ec-b042-ceb2ff65ce3b': 17,
  'aa8a8960-ae08-11ec-aa1f-02ad3a181751': 18,
  'ab3d1080-ae08-11ec-aa10-4f77e1f65932': 19,
  'abdecec0-ae0

Calling marking function for H.

In [11]:
mark_sessions(h_from_excel, H_FILES, questions)

Session 1 of 74
Finished 'pairing' in 1.0272 secs
Session 2 of 74
Finished 'pairing' in 1.0081 secs
Session 3 of 74
Finished 'pairing' in 1.0091 secs
Session 4 of 74
Finished 'pairing' in 0.8339 secs
Session 5 of 74
Finished 'pairing' in 1.0445 secs
Session 6 of 74
Finished 'pairing' in 1.0595 secs
Session 7 of 74
Finished 'pairing' in 0.9857 secs
Session 8 of 74
Finished 'pairing' in 0.9612 secs
Session 9 of 74
Finished 'pairing' in 1.0814 secs
Session 10 of 74
Finished 'pairing' in 1.1062 secs
Session 11 of 74
Finished 'pairing' in 0.0790 secs
Session 12 of 74
Finished 'pairing' in 0.9710 secs
Session 13 of 74
Finished 'pairing' in 1.0319 secs
Session 14 of 74
Finished 'pairing' in 1.1267 secs
Session 15 of 74
Finished 'pairing' in 1.0537 secs
Session 16 of 74
Finished 'pairing' in 0.8741 secs
Session 17 of 74
Finished 'pairing' in 1.0223 secs
Session 18 of 74
Finished 'pairing' in 0.8879 secs
Session 19 of 74
Finished 'pairing' in 0.9608 secs
Session 20 of 74
Finished 'pairing' in 1

{'sMjVKK5bH9bGGEeDF4TMra2xb': {'42da7ff0-ae08-11ec-a14a-d2aa178cb2a7': 0,
  '4d234050-ae08-11ec-bbb2-f9654f54db01': 0,
  '4efa9860-ae08-11ec-b85d-916833758b2d': 1,
  '4ff2db10-ae08-11ec-9e4b-8008a1a26dce': 2,
  '50a0f560-ae08-11ec-8067-87c52775ceb8': 3,
  '51cc07e0-ae08-11ec-aeb8-08d754bcc4cd': 4,
  '526efea0-ae08-11ec-87b8-142015bfd64e': 5,
  '5311ce50-ae08-11ec-bffb-1d2f275cefda': 6,
  '53cc6bc0-ae08-11ec-8a6d-25e09bc98051': 7,
  '549512f0-ae08-11ec-a51a-41c41e391a26': 8,
  '557e3a70-ae08-11ec-b457-debc9c383812': 9,
  '5640ee30-ae08-11ec-a864-77f8dc4ebc9d': 10,
  '56fb3d80-ae08-11ec-99bf-02d0a07774de': 11,
  '57a475d0-ae08-11ec-8ad6-4587f5090d4d': 12,
  '584e7170-ae08-11ec-acc9-81ffa5b7bfdb': 13,
  '58a6f1b0-ae08-11ec-b9ba-b241c8f4e0d7': 13,
  '59de8750-ae08-11ec-89ff-47d640a1cc0c': 14,
  '5ae967a0-ae08-11ec-94fd-2fd0d1222482': 15,
  '5b9ab640-ae08-11ec-8914-00960ebfa8c4': 16,
  '5c3ebe70-ae08-11ec-a7b6-38b6048499e8': 17,
  '5cfbf3f0-ae08-11ec-8d66-f5c30c22c3cf': 18,
  '5e1ef020-ae08